 ## Convert trade log into pnl design matrix for modeling
 Design matrix is one record per row

In [10]:
# imports

import pandas as pd
import numpy as np # for np.nan
import glob # for text matching
import os # for path


In [11]:
# read in raw data
### INPUT ###
attr_filename = 'data/PCM-Tracking - LogHist.csv'

# trade + activity list, max 30 cols
globbed_files = glob.glob('data/*U106*.csv')
col_names_temp = list(range(30))
df_raw = pd.DataFrame(columns = col_names_temp)

# initial date
init_date = '2015-06-30'

for csv in globbed_files:
    frame = pd.read_csv(csv, names=col_names_temp)
    frame['filename'] = os.path.basename(csv)
    df_raw = df_raw.append(frame)

# attributes from trading
df_raw_attr = pd.read_csv(attr_filename)
df_raw_attr['filename'] = os.path.basename(attr_filename)
df_raw_attr = df_raw_attr.append(df_raw_attr)

In [12]:
# Understand df_raw
df_raw.head()


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,filename
0,Statement,Header,Field Name,Field Value,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
1,Statement,Data,BrokerName,Interactive Brokers,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
2,Statement,Data,BrokerAddress,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
3,Statement,Data,Title,Activity Statement,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
4,Statement,Data,Period,"January 1, 2015 - December 31, 2015",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...


In [13]:
# See df_raw available data
df_raw[0].value_counts()[:10]

Borrow Fee Details                                                                                          6089
IBKR Managed Securities Lent Activity (Stock Yield Enhancement Program)                                     5037
Trades                                                                                                      4239
IBKR Managed Securities Lent Activity (Stock Yield Enhancement Program) (Post February 28, 2017)            3604
IBKR Managed Securities Lent Interest Details (Stock Yield Enhancement Program) (Post February 28, 2017)    2467
IBKR Managed Securities Lent Fee Details (Stock Yield Enhancement Program)                                  2305
IBKR Managed Securities Lent Interest Details (Stock Yield Enhancement Program)                             1253
Change in Dividend Accruals                                                                                 1170
Dividends                                                                                       

In [14]:
# Create trading list, after first activity file
df_trades = df_raw[df_raw[0]=='Trades']
df_trades.columns  = df_trades.iloc[0,:]
df_trades.columns = [*df_trades.columns[:-1], 'filename']
cols = df_trades.columns[~df_trades.columns.isin([np.nan])]
df_trades = df_trades[cols]
df_trades = df_trades[df_trades['Header'] == 'Data']
df_trades = df_trades[df_trades['filename'] != os.path.basename(globbed_files[0])]
df_trades.head()

,Trades,Header,DataDiscriminator,Asset Category,Currency,Symbol,Date/Time,Quantity,T. Price,C. Price,Proceeds,Comm/Fee,Basis,Realized P/L,MTM P/L,Code,filename
388,Trades,Data,Order,Stocks,USD,AAPL,"2016-01-11, 10:20:02",200,98.159,98.53,-19631.8,-1,19632.8,2566.569764,74.2,O,F1056881_U1060261_20160104_20161230_AS_Fv2_6b2...
389,Trades,Data,Order,Stocks,USD,AAPL,"2016-01-25, 09:44:44",150,100.4,99.44,-15060,-1,15061,0,-144,O;P,F1056881_U1060261_20160104_20161230_AS_Fv2_6b2...
390,Trades,Data,Order,Stocks,USD,AAPL,"2016-01-27, 12:22:10",-600,95.80105,93.42,57480.63,-3.457643592,-70599.8163,-13122.643944,1428.63,C;LI;P,F1056881_U1060261_20160104_20161230_AS_Fv2_6b2...
391,Trades,Data,Order,Stocks,USD,AAPL,"2016-01-29, 09:51:29",600,95.58,97.34,-57348,-3,57351,13122.643944,1056,O,F1056881_U1060261_20160104_20161230_AS_Fv2_6b2...
392,Trades,Data,Order,Stocks,USD,AAPL,"2016-02-11, 13:48:52",-600,93.17055,93.7,55902.33,-3.818670794,-70473.643944,-14575.132614,-317.67,C;LI;P,F1056881_U1060261_20160104_20161230_AS_Fv2_6b2...


In [15]:
# create initial portfolio based on first activity file
df_port_init = df_raw[df_raw[0]=='Open Positions']
df_port_init.columns  = df_port_init.iloc[0,:]
df_port_init = df_port_init[df_port_init['Header'] == 'Data']
df_port_init.columns = [*df_port_init.columns[:-1], 'filename']
cols = df_port_init.columns[~df_port_init.columns.isin([np.nan])]
df_port_init = df_port_init[cols]

df_port_init = df_port_init[df_port_init['filename'] == os.path.basename(globbed_files[0])]

df_port_init.head()


,Open Positions,Header,DataDiscriminator,Asset Category,Currency,Symbol,Quantity,Mult,Cost Price,Cost Basis,Close Price,Value,Unrealized P/L,Code,filename
281,Open Positions,Data,Summary,Stocks,USD,AAPL,250,1,133.357786144,33339.446536,105.26,26315,-7024.446536,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
282,Open Positions,Data,Summary,Stocks,USD,ABC,500,1,100.714796258,50357.398129,103.71,51855,1497.601871,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
283,Open Positions,Data,Summary,Stocks,USD,ABX,-2000,1,6.424273331,-12848.546663,7.38,-14760,-1911.453337,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
284,Open Positions,Data,Summary,Stocks,USD,AER,600,1,46.97212004,28183.272024,43.16,25896,-2287.272024,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...
285,Open Positions,Data,Summary,Stocks,USD,AGN,40,1,316.4487488,12657.949952,312.5,12500,-157.949952,NaN,F1056881_U1060261_20150102_20151231_AS_Fv2_6ea...


In [16]:
# clean attribute columns

col_dict_attr = {
    'DATE' : 'DATE',
    'CONTRACT' : 'CONTRACT',
    'TIME':'TIME',
    'ACTION':'ACTION',
    'PRICE':'PRICE',
    'QTYCHG':'QTYCHG',
    'COMMISSION':'COMMISSION',
    
    'PCTRETURN': 'PCTRETURN',
}

df_clean_attr = df_raw_attr.copy(deep=True)
df_clean_attr.columns = pd.Series(df_clean_attr.columns.astype(str).str.upper().str.strip())
df_clean_attr.columns = pd.Series(df_clean_attr.columns).map(col_dict_attr)\
    .fillna(pd.Series(df_clean_attr.columns))

df_clean_attr['ACTION'] = df_clean_attr['ACTION'].astype(str).str.strip()

# pull out macro / non trades
df_macro = df_clean_attr[~ df_clean_attr['ACTION'].astype(str).str.contains('BOT') & 
                      ~ df_clean_attr['ACTION'].astype(str).str.contains('SLD') &
                      ~ df_clean_attr['ACTION'].astype(str).str.contains('END')
                      ]
                    
df_clean_attr = df_clean_attr[ df_clean_attr['ACTION'].astype(str).str.contains('BOT') | 
                       df_clean_attr['ACTION'].astype(str).str.contains('SLD') |
                       df_clean_attr['ACTION'].astype(str).str.contains('END')
                        ]

df_clean_attr.head()

,DATE,ACTION,CONTRACT,QTYCHG,PRICE,TIME,UNNAMED: 6,COMMISSION,UNNAMED: 8,CASH CHG (PNL),...,DAYSTOFYEND,FYEPSNXT,GROWTH*0.5TO0.75,ROIC (BW ROA ROE),IMPLIED P/E,YEARS TO NORMALIZATION,LASTUPDATED,CATEGORY,COMMENTS.1,FILENAME
4,5/30/2012,BOT,AIG,500,$29.70,1:30:19 PM,NaN,2.5,NaN,"($14,852.50)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PCM-Tracking - LogHist.csv
5,5/30/2012,BOT,AAPL,12,$569.32,1:36:31 PM,NaN,1.0,NaN,"($6,832.84)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PCM-Tracking - LogHist.csv
6,5/30/2012,SLD,KGC,76,$7.87,1:56:01 PM,NaN,1.0,NaN,$597.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PCM-Tracking - LogHist.csv
7,5/30/2012,SLD,KGC,100,$7.87,1:56:01 PM,NaN,0.0,NaN,$787.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PCM-Tracking - LogHist.csv
8,5/30/2012,SLD,KGC,224,$7.87,1:56:01 PM,NaN,1.0,NaN,"$1,761.88",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PCM-Tracking - LogHist.csv


In [17]:
# update data types for attr

df_clean_attr['DATE'] = pd.to_datetime(df_clean_attr['DATE'],errors='coerce') 
numeric_cols = ['PRICE','COMMISSION','QTYCHG']
for col in numeric_cols:
    df_clean_attr[col] = (df_clean_attr[col].astype(str).str.strip()
        .str.replace('$','').str.replace(',','').astype(float)
        )

# QA
df_clean_attr.dtypes   


DATE                                    datetime64[ns]
ACTION                                          object
CONTRACT                                        object
QTYCHG                                         float64
PRICE                                          float64
TIME                                            object
UNNAMED: 6                                      object
COMMISSION                                     float64
UNNAMED: 8                                      object
CASH CHG (PNL)                                  object
COMMENTS                                        object
PCTRETURN                                       object
DETAILS                                        float64
STARTDATE                                       object
COMPANY NAME (IN ALPHABETICAL ORDER)            object
TICKER                                          object
STOP                                            object
% TO STOP                                       object
CURRENT PR

In [5]:
# initialize pnl 

# create final pnl df
final_cols =                                ['Date_Open', 'Quantity', 'Cost_Basis']
df_complete_trades = pd.DataFrame(columns = final_cols)

# create initial portfolio with key: contract name, 
# item is nested dictionary of qty, cost, date (last updated)
portfolio_dict = {}
for i, row in df_port_init.iterrows():
    portfolio_dict[row['Symbol']] = [init_date, row['Quantity'],  row['Cost Basis']  ]
    


In [ ]:
# update pnl

# for i, row in df_clean_attr.iterrows():
#     if row['ACTION'].find('BOT'):        
#         if row['CONTRACT'] in df_portfolio:
#             # if positive position in portfolio, add to it
#             # if df_portfolio[row['CONTRACT']] > 0:
#             #     df_portfolio[row['CONTRACT']] = (row['PRICE'] * row['QTYCHG'] - row['COMMISSION']
#             #                                     + row['CONTRACT']
#             #                                     )
#             # if negative position
#             # else:
#                 # pass
#         # if not in portfolio, add to it
#         else: 
#             df_portfolio[row['CONTRACT']] = {
#                 'cost': row['PRICE'] * row['QTYCHG'] - row['COMMISSION']
#             }
#     elif row['ACTION'].find('SLD') | row['ACTION'].find('END'):
#         if row['CONTRACT'] in df_portfolio:
#             # df_portfolio[row['CONTRACT']] = (row['CONTRACT'] - (
#             #     row['PRICE'] * row['QTYCHG'] - row['COMMISSION'])
#             #                                  )
#         else: 
#             # ignore errors for now
#             pass
    
#     #QA
#     # print(str(i) +  f'\n' + str(row))
#     if i > 100:
#         break
    